# **Funciones para Endpoints en AWS**

#### **Elegimos realizar nuestras operaciones de Big Data con [`AWS`](https://aws.amazon.com/es/) debido a que es la empresa líder en el sector y cuenta con una alta demanda en la industria. La decisión se basa en la reputación y experiencia sobresalientes de [`AWS`](https://aws.amazon.com/es/) en la gestión eficiente de datos a gran escala. Optar por [`AWS`](https://aws.amazon.com/es/) nos permite aprovechar su infraestructura sólida, servicios avanzados y la confianza que inspira en el mercado, asegurando así que nuestras operaciones de Big Data estén respaldadas por una plataforma de clase mundial reconocida por su fiabilidad y eficacia.**

### Importamos las librerías necesarias para el desarrollo de las funciones.

In [1]:
import boto3
import pandas as pd
import pyarrow.parquet as pq
import pyarrow.fs
from s3fs import S3FileSystem
import s3fs
import joblib
from fastapi import FastAPI, Depends
from pydantic import BaseModel
from datetime import timedelta
from pyathena import connect

### Probamos la conexión con el bucket.

In [32]:
# Especificamos el nombre de tu bucket y el identificador de la región de AWS
bucket_name = 'tu_bucket'
region_name = 'tu_region'
aws_access_key_id = 'tu_access_key_id'
aws_secret_access_key = 'tu_secret_access_key'

# Creamos un cliente de S3 con las credenciales proporcionadas
s3 = boto3.client('s3', region_name=region_name, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# Listamos objetos en el bucket
response = s3.list_objects(Bucket=bucket_name)
for obj in response.get('Contents', []):
    print(f'Archivo: {obj["Key"]}, Tamaño: {obj["Size"]} bytes')

Archivo: air_quality.parquet, Tamaño: 4692 bytes
Archivo: climate_NY.parquet, Tamaño: 14001 bytes
Archivo: electric_and_alternative_fuel_charging_stations.parquet, Tamaño: 14938 bytes
Archivo: fuel_prices.parquet, Tamaño: 2498 bytes
Archivo: sound_quality.parquet, Tamaño: 9825 bytes
Archivo: taxi_zones.parquet, Tamaño: 7501 bytes
Archivo: taxis_2023.parquet, Tamaño: 515237183 bytes
Archivo: vehicles_info.parquet, Tamaño: 9813 bytes
Archivo: xgb_model.pkl, Tamaño: 422934 bytes


### Revisamos el contenido de un archivo del bucket al azar.

In [18]:
# Especificamos las credenciales de AWS
aws_access_key_id = 'tu_access_key_id'
aws_secret_access_key = 'tu_secret_access_key'

# Especificamos la ruta de tu archivo Parquet en S3
s3_path = 'tu_bucket/tu_archivo'

# Configuramos la conexión a S3 con las credenciales
s3 = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)

# Abrimos el archivo Parquet directamente desde S3
parquet_file = pq.ParquetFile(s3.open(s3_path))

# Leemos el archivo Parquet en un DataFrame de pandas
df = parquet_file.read().to_pandas()

# Imprimimos los resultados
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Year                 74 non-null     int64 
 1   Manufacturer         74 non-null     object
 2   Model                74 non-null     object
 3   Fuel                 74 non-null     object
 4   Alternative Fuel     74 non-null     object
 5   Miles per gallon     74 non-null     int64 
 6   CO2 Emission (g/mi)  74 non-null     int64 
 7   Sound Emission (dB)  74 non-null     int64 
 8   Price (USD)          74 non-null     int64 
 9   Range (mi)           74 non-null     int64 
 10  Category             74 non-null     object
dtypes: int64(6), object(5)
memory usage: 6.5+ KB
None


#### `Función que devuelve la predicción porcentual de oportunidad de viajes de un usuario.`

In [81]:
class InputData(BaseModel):
    pBoroughID: int
    dayofweek: int
    hour: int


def predict(data: InputData):

    # Especificamos las credenciales de AWS
    aws_access_key_id = 'tu_access_key_id'
    aws_secret_access_key = 'tu_secret_access_key'

    # Configuramos la conexión a S3 con las credenciales
    s3 = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)

    # Especificamos la ruta del modelo en S3
    model_path = 'tu_bucket/tu_modelo'

    # Descargamos el modelo desde S3
    with s3.open(model_path, 'rb') as f:
        xgb_model = joblib.load(f)

    # Convertirmos los datos de entrada a un DataFrame
    input_data = pd.DataFrame([data.model_dump()])

    # Realizarmos la predicción
    prediction = xgb_model.predict(input_data)

    # Devolvemos la predicción como JSON
    return {"Probabilidad de conseguir pasajero": prediction.tolist()}

In [84]:
input_data = InputData(pBoroughID=1, dayofweek=5, hour=18)
predict(input_data)

{'Probabilidad de conseguir pasajero': [79.81958770751953]}

#### `Función que devuelve un top 3 de vehículos ecológicos de acuerdo al capital ingresado`

In [70]:
def top_3_vehicles(max_price_usd: float):

    # Especificamos las credenciales de AWS
    aws_access_key_id = 'tu_access_key_id'
    aws_secret_access_key = 'tu_secret_access_key'

    # Especificamos la ruta del archivo Parquet en S3
    s3_path = 'tu_bucket/tu_archivo'

    # Configura la conexión a S3 con las credenciales
    s3 = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)

    # Abrimos el archivo Parquet directamente desde S3
    parquet_file = pq.ParquetFile(s3.open(s3_path))

    # Leemos el archivo Parquet en un DataFrame de pandas
    df = parquet_file.read().to_pandas()

    # Filtramos los vehículos por precio
    filtered_vehicles = df[df['Price (USD)'] <= max_price_usd]

    # Ordenamos los vehículos por CO2 en orden ascendente, luego por Sound Emission y finalmente por Range
    sorted_vehicles = filtered_vehicles.sort_values(by=['CO2 Emission (g/mi)', 'Sound Emission (dB)', 'Range (mi)'], ascending=[True, True, False])

    # Tomamos los tres primeros vehículos dentro del rango de precio
    top_3_vehicles = sorted_vehicles.head(3)

    # Creamos el formato de salida como una lista de diccionarios
    output_format = []
    for idx, row in enumerate(top_3_vehicles.itertuples(), start=1):
        output_format.append({
            f'Puesto {idx}': f'{row.Manufacturer} {row.Model}',
            'Precio (USD)': row._9,
            'Combustible': row.Fuel,
            'CO2': row._7,
            'dB': row._8,
            'Millas con un tanque lleno': row._10,
        })

    return output_format

In [72]:
top_3_vehicles(25500)

[{'Puesto 1': 'Chevrolet Equinox Hybrid',
  'Precio (USD)': 24995,
  'Combustible': 'Gasoline',
  'CO2': 25,
  'dB': 68,
  'Millas con un tanque lleno': 440},
 {'Puesto 2': 'Toyota Prius',
  'Precio (USD)': 25295,
  'Combustible': 'Gasoline',
  'CO2': 41,
  'dB': 65,
  'Millas con un tanque lleno': 54},
 {'Puesto 3': 'Toyota Prius',
  'Precio (USD)': 24525,
  'Combustible': 'Gasoline',
  'CO2': 41,
  'dB': 68,
  'Millas con un tanque lleno': 520}]

#### `Función que devuelve estadísticas de viajes de acuerdo al Borough ingresado`

In [29]:
def taxi_stats_test(pickup_borough: str):
    # Configuramos las credenciales de AWS
    aws_access_key_id = 'tu_access_key_id'
    aws_secret_access_key = 'tu_secret_access_key'
    region_name = 'tu_region'

    # Configuramos la conexión a Athena
    conn = connect(aws_access_key_id=aws_access_key_id,
                aws_secret_access_key=aws_secret_access_key,
                region_name=region_name,
                schema_name='tu_schema',
                s3_staging_dir='tu_bucket')

    # Ejecutamos una consulta
    query = '''
        WITH tu_table AS (
    SELECT
        "pickup_borough",
        COUNT(*) AS "Viajes_Totales",
        ROUND(AVG(date_diff('second', "pickup_datetime", "dropoff_datetime")) / 3600, 2) AS "Duracion_Promedio_Hs",
        ROUND(COUNT(*) / COUNT(DISTINCT date_trunc('day', "pickup_datetime")), 2) AS "Media_de_Viajes_por_Dia",
        ROUND(COUNT(*) / COUNT(DISTINCT date_trunc('month', "pickup_datetime")), 2) AS "Media_de_Viajes_por_Mes",
        ROUND(AVG("trip_distance"), 2) AS "Distancia_recorrida_promedio_millas",
        ROUND(AVG("total_amount"), 2) AS "Total_ganado_en_promedio_USD"
    FROM
        "tu_database"."tu_table"
    GROUP BY
        "pickup_borough"
    )
    SELECT
    *
    FROM
    tu_table
    ORDER BY
    "pickup_borough"
    '''

    cursor = conn.cursor()
    cursor.execute(query, parameters={'pickup_borough': 'Manhattan'})  

    result = pd.DataFrame(cursor.fetchall(), columns=["pickup_borough", "Viajes_Totales", "Duracion_Promedio_Hs", "Media_de_Viajes_por_Dia", "Media_de_Viajes_por_Mes", "Distancia_recorrida_promedio_millas", "Total_ganado_en_promedio_USD"])

    borough_result = result[result["pickup_borough"] == pickup_borough]

    formatted_result = {
        "Viajes Totales": borough_result["Viajes_Totales"].values[0],
        "Duración Promedio (Hs)": borough_result["Duracion_Promedio_Hs"].values[0],
        "Media de Viajes por Día": borough_result["Media_de_Viajes_por_Dia"].values[0],
        "Media de Viajes por Mes": borough_result["Media_de_Viajes_por_Mes"].values[0],
        "Distancia recorrida promedio (millas)": borough_result["Distancia_recorrida_promedio_millas"].values[0],
        "Total ganado en promedio (USD)": borough_result["Total_ganado_en_promedio_USD"].values[0]
    }

    # En el entorno de Athena, enfrentamos un inconveniente al realizar consultas desde Python, ya que dichas consultas generan
    # automáticamente dos archivos en el bucket: uno con extensión .csv y otro con extensión .metadata. Esta situación impide la
    # ejecución de nuevas consultas hasta que ambos archivos sean eliminados. Para superar este problema, hemos implementado una
    # solución que conecta con S3 añadiendo las siguientes líneas de código. Estas líneas permiten la eliminación de los archivos generados,
    # posibilitando así la ejecución de consultas sin inconvenientes.
    
    s3_client = boto3.client('s3',
                            aws_access_key_id=aws_access_key_id,
                            aws_secret_access_key=aws_secret_access_key,
                            region_name=region_name)

    # Definimos el nombre del bucket
    bucket_name = 'tu_bucket'

    # Obtenemos la lista de objetos en el bucket
    response = s3_client.list_objects_v2(Bucket=bucket_name)

    # Eliminamos los archivos .csv y .metadata encontrados
    for obj in response['Contents']:
        if obj['Key'].endswith('.csv') or obj['Key'].endswith('.metadata'):
            s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])


    return formatted_result

In [31]:
taxi_stats_test('Manhattan')

{'Viajes Totales': 24235349,
 'Duración Promedio (Hs)': 0.25,
 'Media de Viajes por Día': 85940,
 'Media de Viajes por Mes': 1425608,
 'Distancia recorrida promedio (millas)': 3.24,
 'Total ganado en promedio (USD)': 26.35}